# Dark and bias analysis

The input consists of sets of 31 dark frames and 31 bias frames. One set was taken with the same settings used for the light exposures; the other set with settings identical with the flat exposures:

 - Darks and bias for the ligth exposures: ISO 6400 and 12800, exposure time 3.2s
 - Darks and bias for the flat exposures: ISO 100, exposure 1/40s

Bias in both sets were taken at 1/8000s. The darks were taken on-site right after the ligths. Bias taken next day. 

In [ ]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import exifread
import rawpy

## Utility functions 

In [ ]:
# Stats
def stats(array, x, y, size, name, results):
    slice_x = slice(x,x+100)
    slice_y = slice(y,y+100)

    mean = np.mean(array[slice_x,slice_y])
    std = np.std(array[slice_x,slice_y])
    
    results[name] = [mean, std]
    print(name, "  mean =", mean, "stddev =", std)

In [ ]:
# Combine darks from list, using a provided numpy averaging function.
def combine_arrays(file_list, combination_function=np.average):
    stack = None
    for fname, i in zip(file_list, range(len(file_list))):

        raw = rawpy.imread(fname)
        array = raw.raw_image_visible

        if stack is None:            
            stack = np.zeros(shape=(array.shape[0],array.shape[1],len(file_list)), dtype='float32')

        # Actual raw DN values are obtained by subtracting the
        # camera-created black level per channel. In the Sony A7, this
        # parameter is the same for all channels, and constant.
        # Just subtract it from everything.
        stack[:,:,i] = np.array(array, dtype='float32') - 512.

    return combination_function(stack, axis=2)

In [ ]:
def process(path, title, results, vmin=-8, vmax=4, combination_function=np.average, output=False):
    list_p = glob.glob(path + '/*.ARW')

    # get exposure time
    f = open(list_p[0], 'rb')
    tags = exifread.process_file(f)
    exptime = str(tags['EXIF ExposureTime'])    
    
    array = combine_arrays(list_p, combination_function=combination_function)

    results[title] = {}
    
    x = int(array.shape[0] / 2)
    y = int(array.shape[1] / 2)
    stats(array, x, y, size, "Center:", results[title])

    x = 10
    y = 10
    stats(array, x, y, size, "Corner:", results[title])

    plt.figure(figsize=[10, 6])
    plt.imshow(array, vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.title(title + " " + exptime)
    
    if output:
        return array

## Define paths to data files

In [ ]:
path1 = '../astrophotography_data/MilkyWayPrettyBoy/12800/'
path2 = '../astrophotography_data/MilkyWayPrettyBoy/6400/'
path3 = '../astrophotography_data/MilkyWayPrettyBoy/darks/ISO100_3.2s/'
path4 = '../astrophotography_data/MilkyWayPrettyBoy/darks/ISO3200_25s/'

## Define dictionary to store summary of results

In [ ]:
results = {}

## Analysis of dark/bias for light exposures

#### ISO 12800

In [ ]:
dark_path = os.path.join(path1,'dark')
process(dark_path, 'Average dark - ISO 12800 31 frames', results)

In [ ]:
bias_path = os.path.join(path1,'bias')
process(bias_path, 'Average bias - ISO 12800 31 frames', results)

In [ ]:
process(dark_path, 'Dark standard deviation - ISO 12800 31 frames', results,
        vmin=9, vmax=14, combination_function=np.std)

In [ ]:
process(bias_path, 'Bias standard deviation - ISO 12800 31 frames', results,
        vmin=9, vmax=14, combination_function=np.std)

#### ISO 6400

In [ ]:
dark_path = os.path.join(path2,'dark')
process(dark_path, 'Average dark - ISO 6400 31 frames', results)

In [ ]:
bias_path = os.path.join(path2,'bias')
process(bias_path, 'Average bias - ISO 6400 31 frames', results)

In [ ]:
process(dark_path, 'Dark standard deviation - ISO 6400 31 frames', results,
        vmin=5, vmax=8, combination_function=np.std)

In [ ]:
process(bias_path, 'Bias standard deviation - ISO 6400 31 frames', results,
        vmin=5, vmax=8, combination_function=np.std)

## Analysis of dark/bias for flat exposures (ISO 100)

In [ ]:
dark_path = os.path.join(path3,'dark')
process(dark_path, 'Average dark - ISO 100 31 frames', results,
       vmin=0.6, vmax=1.4)

In [ ]:
bias_path = os.path.join(path3,'bias')
process(bias_path, 'Average bias - ISO 100 31 frames', results,
        vmin=0.8, vmax=1.8,)

In [ ]:
process(dark_path, 'Dark standard deviation - ISO 100 31 frames', results,
        vmin=2.0, vmax=2.4, combination_function=np.std)

In [ ]:
process(bias_path, 'Bias standard deviation - ISO 100 31 frames', results,
        vmin=2.0, vmax=2.4, combination_function=np.std)

In [ ]:
for k1 in results:
    print(k1)
    r = results[k1]
    for k2 in r:
        print(k2, r[k2])
    print()

## Conclusions

##### For short exposures!

- Read noise

   Bias variance for any given pixel, along a sequence of bias exposures:
     - ISO   100 - 2.2 DN
     - ISO  6400 - 6.3 DN
     - ISO 12800 -  12 DN
 
   These are large when compared with the across-detector bias variance in the averaged frames - thus should be a good    estimator of the sensor readnoise.


- Dark fixed pattern

   In a similar way as above for the read out noise, the frame-to-frame dark variance for any given pixel is large when compared with the across-detector dark variance in the averaged darks. This suggest that no significant high-frequency fixed pattern exists in the dark signal. 



Darks have low mean values at any ISO, but the variance increases rapidily with increasing ISO. The low mean value seemingly independent of the ISO setting suggests that the camera hardware is calibrated to adjust the ADC dark level based on the sensor gain setting. 

# Looking for pattern in long exposure dark frames

Using the minimum entropy method to find the optimal damping factor for the dark frame (https://www.cs.ubc.ca/labs/imager/tr/2001/goesele2001a/goesele.2001a.pdf)

## Data

In [ ]:
dark_path = os.path.join(path4,'dark')
dark_average = process(dark_path, 'Average dark - ISO 3200 30 frames', results, 
                       output=True, vmin=15.5, vmax=19.5)

In [ ]:
process(dark_path, 'Dark standard deviation - ISO 3200 30 frames', results, combination_function=np.std,
       vmin=7., vmax=9.)

## Entropy

In [ ]:
def entropy(target_array, dark_array, nbins=30, y1=None, y2=None):
    x = []
    y = []
    for k in arange(0., 1.8, 0.02):
        # Subtract dark from target.
        dark_subtracted = target_array - k * dark_array

        # Build frequency density histogram.
        hist = np.histogram(dark_subtracted, bins=nbins, density=True)
        hist = hist[0] #/ float(target_array.shape[0] * target_array.shape[1])

        # Compute entropy.
        logp = np.log10(hist)
        s = -np.sum(hist * logp)

        x.append(k)
        y.append(s)

    plt.figure(figsize=[10, 6])
    if y1 is not None and y2 is not None:
        plt.ylim(y1, y2)
    plt.scatter(x, y)

In [ ]:
# ISO 3200 - MW image, average dark
basepath_dark = '../astrophotography_data/MilkyWayPrettyBoy/darks'
basepath_target = '../astrophotography_data/assateague/milky_way'

fullpath = os.path.join(basepath_target, 'raw')
filename = os.path.join(fullpath, 'DSC03288.ARW')
raw = rawpy.imread(filename)
target_array = raw.raw_image_visible - 512.

fullpath = os.path.join(basepath_dark, 'ISO3200_25s/dark')
filename = os.path.join(fullpath, 'DSC07112.ARW')
raw = rawpy.imread(filename)
dark_array = raw.raw_image_visible - 512.

entropy(target_array, dark_array)
# entropy(target_array, dark_average, y1=0.0053, y2=0.006)

In [ ]:
# ISO 3200 - target image, average dark
basepath_dark = '../astrophotography_data/MilkyWayPrettyBoy/darks'
basepath_target = '../astrophotography_data/MilkyWayPrettyBoy/darks'

fullpath = os.path.join(basepath_target, 'target_entropy')
filename = os.path.join(fullpath, 'f11.ARW')
raw = rawpy.imread(filename)
target_array = raw.raw_image_visible - 512.

entropy(target_array[700:2000,1500:3000,], dark_average[700:2000,1500:3000,], nbins=100)
# plt.figure(figsize=[10, 6])
# plt.imshow(target_array[700:2000,1500:3000,])

## Conclusion

Dark subtraction does not affect the entropy significantly. We can make do with just subtracting 512 from each pixel. 

It remains to be seen if this is indeed the best approach. A test running a processing pipeline with and without dark subtraction can solve the problem.